In [1]:
# Load Word2Vec model
from gensim.models import Word2Vec as w2v

FILE = "C:/Users/MyPC/Desktop/Vegito/W2V Models/w2v_reddit_unigram_300d.bin"
model = w2v.load_word2vec_format(FILE, binary=True)

C:\Users\MyPC\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\MyPC\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
# Load the dataset
import pandas as pd

FILE = "C:/Users/MyPC/Desktop/Vegito/clean_dataset.csv"
df = pd.read_csv(FILE)
len(df)

6580

In [3]:
# Get the unique words in the dataset
unique_words = []

for comment in df['Comment']:
    
    for word in comment.split():
        if word not in unique_words:
            unique_words.append(word)
            
len(unique_words)

19286

In [4]:
# Check how many are not in the reddit model and collect them
not_in_model = []

for word in unique_words:
    
    if word not in model:
        not_in_model.append(word)
        
print("NUMBER OF WORDS NOT IN MODEL: ", len(not_in_model))
print("PERCENTAGE MISSING: ", (len(not_in_model)/len(unique_words)) * 100)

NUMBER OF WORDS NOT IN MODEL:  1141
PERCENTAGE MISSING:  5.916208648760759


In [21]:
# Sort the words then print them. Then, write them in a text file (DONE)
#not_in_model = sorted(not_in_model)
#with open('Missing words.txt', 'w') as fh:
#    for word in not_in_model:
#        fh.write("{}\n".format(word))

In [6]:
# Get the number of words in the Word2Vec model
print(len(model.syn0))

1146604


In [7]:
# Get words that are similar. This returns tuples in a list
word = 'asshole'
top_n = 10

similar_words = model.most_similar(word, topn=top_n)
model.most_similar(word, topn=top_n)

[('arsehole', 0.8830243349075317),
 ('asshat', 0.8711205124855042),
 ('prick', 0.8389500975608826),
 ('dickhead', 0.8373932242393494),
 ('douchebag', 0.8241000175476074),
 ('dickbag', 0.8229827880859375),
 ('ahole', 0.8007912039756775),
 ('idiot', 0.7848369479179382),
 ('jackass', 0.7815600037574768),
 ('douche', 0.7751066088676453)]

In [8]:
# Calculate the Mean Cosine similarity among words
import numpy as np

mean_cos_distance = np.mean([ cos_distance for word, cos_distance in similar_words ])

In [9]:
# Get the collected words that are similar above this score
words_above_mean = [word for word, cos_distance in similar_words if cos_distance > mean_cos_distance]
total_words = float(len(words_above_mean))

In [10]:
words_above_mean

['arsehole', 'asshat', 'prick', 'dickhead', 'douchebag', 'dickbag']

In [11]:
# Pre-initialize an empty numpy array (for speed)
avgWordsFeature = np.zeros((300,),dtype="float32")

In [12]:
# Loop over each word
for word in words_above_mean:

    # Add the word's vector
    avgWordsFeature = np.add(avgWordsFeature,model[word])
    
# Average them out
avgWordsFeature = np.divide(avgWordsFeature,total_words)

In [13]:
# Use a numpy array to store each word as a feature
# Use the first N-words. N is an integer

sentenceWordFeature = np.zeros((5,), dtype="float32")
print(sentenceWordFeature)

[ 0.  0.  0.  0.  0.]


In [14]:
# Test with a single sentence

sentence = "you goddamn bastard"
complete_sentence = 1 # 1 for full, 0 for incomplete

for i,word in enumerate(sentence.split()):
    
    if i == len(sentenceWordFeature):
        complete_sentence = 0
        break
        
    if word in model:
        word_feature = np.mean(model[word])
    else:
        word_feature = -1.0
        
    sentenceWordFeature[i] = word_feature
    
    print(sentenceWordFeature)

[-0.00470508  0.          0.          0.          0.        ]
[-0.00470508  0.00158833  0.          0.          0.        ]
[-0.00470508  0.00158833 -0.00082337  0.          0.        ]


In [39]:
# Soundex Dictionary (Normalized)
soundex_dictionary = { 
    'B': "1", "F": "1", "P": "1",
    "V": "1", "C": "2", "G": "2",
    "J": "2", "K": "2", "Q": "2",
    "S": "2", "X": "2", "Z": "2", 
    "D": "3", "T": "3", "L": "4", 
    "M": "5", "N": "5", "R": "6", 
    "A": ".", "E": ".", "I": ".",
    "O": ".", "U": ".", "Y": "."
}

In [40]:
 def getSoundex(word):
    
    # Uppercase the word
    word = word.upper()

    # Get the first letter of the word
    soundex = word[0]

    # Skip the following letters
    skip_dict = "HW"
    word = [letter for letter in word[1:] if letter not in skip_dict]
    word = "".join(word)

    # Loop character by character (Start with 2nd character)
    for char in word[0:]:

        code = soundex_dictionary[char]

        if code != soundex[-1]:
            soundex += code

    # Replace period characters
    soundex = soundex.replace(".", "")

    # If the string has only one character, append rest with three 0s.
    soundex = soundex[:4].ljust(4, "0")

    return soundex

In [41]:
getSoundex('fucked')

'F230'

In [17]:
# Load Soundex dictionary
import pickle

FILE = "C:/Users/MyPC/Desktop/Vegito/Word Dictionaries/soundex_dict.pk"

soundex_dict = pickle.load(open(FILE,"rb"))

In [52]:
# Perform operations here
print("TOTAL WORDS: %i" % (len(soundex_dict)))

unique_soundex = list(set([value for key, value in soundex_dict.items()]))

print("UNIQUE SOUNDEX VALUES: %i" % (len(unique_length)))

print("PERCENTAGE: ", (len(unique_length)/len(soundex_dict)) * 100)

TOTAL WORDS: 1146604
UNIQUE SOUNDEX VALUES: 6599
PERCENTAGE:  0.5755256391919092


In [35]:
# Load in missing word text file
FILE = "C:/Users/MyPC/Desktop/Vegito/Text Files/Missing words.txt"
missing_words = []

with open(FILE, 'r') as fh:
    
    # Iterate line by line
    # Remove new line characters
    for line in fh:
        missing_words.append(line.strip())

In [58]:
count = 0

for word in missing_words:
    
    soundex_encode = getSoundex(word)
    
    if soundex_encode in unique_soundex:
        index = unique_soundex.index(soundex_encode)
        print(word, soundex_encode, index)
        


abuseroffacts A126 6143
aciddragon A236 5813
acknowledgs A254 2161
adolescentu A342 3061
adumbas A351 3422
afixing A125 3091
afrocraft A162 1902
afvet A130 789
aguycommenting A225 6414
aidr A360 1130
aitarz A362 6106
aitazaz A322 2916
aitn A350 2775
ajajajajajajaka A222 5773
ajue A200 186
ajumoke A252 1644
aktu A230 5509
akuatuta A233 2722
akuntansi A253 5646
alakda A423 782
albernen A416 5124
aldasity A432 1606
aldickweed A432 1606
aldickweeds A432 1606
aleging A425 4248
alejate A423 782
alenlemone A454 1872
alisterdog A423 782
aljazeerahreader A422 3983
allahpundit A415 376
allinit A453 4402
allra A460 4712
almajiris A452 2649
alsum A425 4248
ameliemaryann A545 1759
americansyrians A562 5096
americanvoters A562 5096
ammede A530 658
amongi A552 221
anaransol A565 3482
andignorant A532 5227
annagillmodel A524 852
annavaram A516 2392
annelli A540 6226
anoellment A545 1759
anonymouslulz A555 3102
antaga A532 5227
antoniososa A535 1185
antunnaru A535 1185
apeboy A110 5898
applegadget A142